In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from swift.llm import get_model_tokenizer, load_dataset, get_template, EncodePreprocessor
from swift.utils import get_logger, find_all_linears, get_model_parameter_info, plot_images, seed_everything
from swift.tuners import Swift, LoraConfig

from swift.trainers import Seq2SeqTrainer, Seq2SeqTrainingArguments

from functools import partial


logger = get_logger()
seed_everything(42)

In [ ]:
model, tokenizer = get_model_tokenizer('Qwen/Qwen2-7B-Instruct')
logger.info(f'model_info: {model.model_info}')
template = get_template(model.model_meta.template, tokenizer)
template.set_mode('train')

target_modules = find_all_linears(model)
lora_config = LoraConfig(task_type='CAUSAL_LM', r=8, target_modules=target_modules)
model = Swift.prepare_model(model, lora_config)
logger.info(f'lora_config: {lora_config}')
logger.info(f'model: {model}')

model_parameter_info = get_model_parameter_info(model)
logger.info(f'model_parameter_info: {model_parameter_info}')

In [ ]:
dataset = ['AI-ModelScope/alpaca-gpt4-data-zh#500', 'AI-ModelScope/alpaca-gpt4-data-en#500',
           'swift/self-cognition#500']

train_dataset, val_dataset = load_dataset(dataset, split_dataset_ratio=0.01, num_proc=4,
        model_name=['小黄', 'Xiao Huang'], model_author=['魔搭', 'ModelScope'])

logger.info(f'train_dataset: {train_dataset}')
logger.info(f'val_dataset: {val_dataset}')
logger.info(f'train_dataset[0]: {train_dataset[0]}')

train_dataset = EncodePreprocessor(template=template)(train_dataset, num_proc=4)
val_dataset = EncodePreprocessor(template=template)(val_dataset, num_proc=4)
logger.info(f'encoded_train_dataset[0]: {train_dataset[0]}')

template.print_inputs(train_dataset[0])

In [ ]:
output_dir = 'output/Qwen2-7B-Instruct'
logger.info(f'output_dir: {output_dir}')
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    logging_steps=5,
    gradient_checkpointing=True,
    weight_decay=0.1,
    lr_scheduler_type='cosine',
    report_to=['tensorboard'],
    logging_first_step=True,
    save_strategy='steps',
    save_steps=50,
    eval_strategy='steps',
    eval_steps=50,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    metric_for_best_model='loss',

)
model.enable_input_require_grads()
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=template.data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    template=template,
)
trainer.model_accepts_loss_kwargs = True
trainer.train()

In [ ]:
logger.info(f'last_model_checkpoint: {trainer.state.last_model_checkpoint}')

images_dir = os.path.join(training_args.output_dir, 'images')
logger.info(f'images_dir: {images_dir}')
plot_images(images_dir, training_args.logging_dir, ['train/loss'], 0.9)